In [149]:
%pip install openai
%pip install chess
%pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [150]:
# import required libraries
import os
import openai
import json
import numpy as np
import pandas as pd
import chess
import copy
import time

from ast import literal_eval
from dotenv import load_dotenv, find_dotenv
from sklearn.model_selection import train_test_split


In [151]:
# import environment variable into notebook
_ = load_dotenv(find_dotenv())  # read local .env file
openai.api_key = os.getenv('OPENAI_API_KEY_UPV')


In [152]:
# list openai available models.
ids = [item["id"] for item in (openai.Model.list())['data']]
print(ids)


['whisper-1', 'babbage', 'text-davinci-003', 'davinci', 'text-davinci-edit-001', 'babbage-code-search-code', 'text-similarity-babbage-001', 'code-davinci-edit-001', 'text-davinci-001', 'ada', 'babbage-code-search-text', 'babbage-similarity', 'gpt-3.5-turbo-16k-0613', 'code-search-babbage-text-001', 'text-curie-001', 'gpt-3.5-turbo-0301', 'gpt-3.5-turbo-16k', 'code-search-babbage-code-001', 'text-ada-001', 'text-similarity-ada-001', 'curie-instruct-beta', 'ada-code-search-code', 'ada-similarity', 'code-search-ada-text-001', 'text-search-ada-query-001', 'davinci-search-document', 'ada-code-search-text', 'text-search-ada-doc-001', 'davinci-instruct-beta', 'text-similarity-curie-001', 'code-search-ada-code-001', 'ada-search-query', 'text-search-davinci-query-001', 'curie-search-query', 'davinci-search-query', 'babbage-search-document', 'ada-search-document', 'text-search-curie-query-001', 'text-search-babbage-doc-001', 'curie-search-document', 'text-search-curie-doc-001', 'babbage-search-q

In [153]:
def request(
    prompt="",
    model="gpt-3.5-turbo-16k",
    temperature=0,
    context=np.array([])
):
    """
    Helper function to make a request to openai api, and return it's response.
    args:
        prompt (string)
        model (str)
        temperature (int)
    returns:
        str
    """
    context = np.append(
        context,
        np.array([{
            "role": 'user',
            "content": f"""
                {prompt}
            """,
        }]), axis=0)

    response = openai.ChatCompletion.create(
        model=model,
        messages=context.tolist(),
        temperature=temperature,  # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]


In [154]:
# create dataframe from downloaded dataset
df = pd.read_csv('data/db.csv')
# create dataframe of one move chess puzzles
df_one = df[df['Themes'].str.contains('mateIn1')].reset_index(drop=True)


In [155]:
# train_df_one, test_df_one = train_test_split(
#     df_one, test_size=0.9999, random_state=42)

# train_df_one = train_df_one.reset_index(drop=True)
# test_df_one = test_df_one.reset_index(drop=True)

# print(len(train_df_one))


In [156]:
def push_move(board: chess.Board, move: chess.Move):
    """ 
    args:
        board (chess.Board)
        moves (chess.Move)
    """
    if board.is_legal(move):
        board.push(move)
    return board


def is_legal_move(board: chess.Board, move: chess.Move) -> bool:
    """ 
    Function that given a fen an a move checks
    if the provided move is legal
    args:
        board (chess.Board)
        moves (str)
    returns:
        bool  
    """
    return board.is_legal(move)


In [157]:
def process_test_row(row, context):
    # create chess board with 'fen' dataset
    board = chess.Board(row['FEN'])
    # create chess moves from 'uci' moves from dataset
    chess_moves = [chess.Move.from_uci(move) for move in row['Moves'].split()]
    # retrieve last move
    chess_moves.pop()
    
    # add all moves except the last one.
    for move in chess_moves:
        push_move(board=board, move=move)
        
    # generate prompt
    prompt = f"""
        "fen": {board.fen()}
        "valid_moves": { json.dumps([move.uci() for move in list(board.legal_moves)])}
    """

    # req to gpt model
    req = request(prompt=prompt, context=context)
    res = literal_eval(req)
    
    # check if valid gpt res move
    valid_res = is_legal_move(board=board, move=chess.Move.from_uci(res['move']))

    # info printings
    # print(f"[------{row['PuzzleId']}------]")
    # print(f"[gpt-move | dataset-move] : {res['move']} | {last_move}")

    if (valid_res is not True):
        print(f"[valid-move] : {valid_res}")
        return

    board_gpt = push_move(board=copy.deepcopy(board), move=chess.Move.from_uci(res['move']))
    print(f"[{row['PuzzleId']}][gpt-completed]: { board_gpt.is_checkmate()}")


In [158]:
def process_training_row(row):
    board = chess.Board(row['FEN'])
    move = chess.Move.from_uci(row['Moves'].split()[0])
    board.push(move)

    valid_moves = list(board.legal_moves)

    return np.array([
        {
            "role": "user",
            "content": json.dumps({
                "fen": board.fen(),
                "valid_moves": [move.uci() for move in valid_moves]
            })
        }, {
            "role": "assistant",
            "content": json.dumps({
                "move": row['Moves'].split()[1]
            })
        }
    ])


In [159]:
context = np.array([])

# for index, row in train_df_one.iterrows():
#     context = np.append(context, process_training_row(row))

messages = np.array([
    {
        "role": "system",
        "content": f"""      
                You are a chess engine playing that solves chess puzzles. \
                Your main task will be to solve chess puzzles.
                
                Therefore, I will provide you with board position in \
                'FEN (Forsyth–Edwards Notation)' format. Also all the \
                legal moves available for that FEN board in UCI (Universal \
                Chess Interface) format.
                
                The prompt input structure will be something like this:
                
                ```
                    "fen": <fen>
                    "valid_moves" : <valid_moves>
                ```
                
                Your task is to choose one of the moves from the legal \
                moves list provided in the prompt that produces a mate \ 
                in one to the provided FEN
                                
                Please do not provide any extra definition or information, just output \
                the movement in a JSON Object with the following format:
                
                ```
                    "move": <your_move>
                ```
            """,
    },
])

context = np.append(messages, context)


In [160]:
checkmate_df = pd.DataFrame(columns=df.columns)

for index, row in df_one.iterrows():
    board = chess.Board(row['FEN'])
    chess_moves = [chess.Move.from_uci(move) for move in row['Moves'].split()]

    for move in chess_moves:
        if (move in board.legal_moves):
            push_move(board=board, move=move)

    if (board.is_checkmate()):
        checkmate_df = pd.concat([checkmate_df, row.to_frame().T])
    else:
        print(row['PuzzleId'])


In [161]:
for index, row in checkmate_df.iterrows():
    process_test_row(row, context)
    if (index == 100):
        break
    time.sleep(0.5)


[jsGhG][gpt-completed]: True
[6Hlem][gpt-completed]: True
[U3gT1][gpt-completed]: False
[MkUOG][gpt-completed]: False
[kaXL5][gpt-completed]: False
[m71OP][gpt-completed]: False
[oQtjY][gpt-completed]: False
[ad0Un][gpt-completed]: False
[kmzKl][gpt-completed]: True
[unMJm][gpt-completed]: False
[dnwzp][gpt-completed]: False
[LisXP][gpt-completed]: False
[zVQX8][gpt-completed]: False
[ZEVIc][gpt-completed]: False
[nCARu][gpt-completed]: False
[MOwON][gpt-completed]: False
[d20aq][gpt-completed]: False
[VSqRq][gpt-completed]: False
[R0XGC][gpt-completed]: False
[NY31d][gpt-completed]: False
[aZZLS][gpt-completed]: False
[gjNcB][gpt-completed]: True
[6qmZO][gpt-completed]: False
[RMOT4][gpt-completed]: False
[aZBX5][gpt-completed]: False
[R1vr6][gpt-completed]: True
[c3Ukn][gpt-completed]: False
[X6XsH][gpt-completed]: False


KeyboardInterrupt: 